# Benchmarks

## Initialize

In [6]:
#library(Rmisc)
library(dtplyr)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)
library(data.table)
library("jsonlite")
library(ggthemes)

In [14]:
base_path = "/home/jakobs"

project_path = glue("{base_path}/data")
experiment = '230323'
experiment_path = glue("{project_path}/{experiment}")

endpoints_md = endpoint_defs = arrow::read_feather(glue("{base_path}/data/endpoints_epic_md.feather")) %>% filter(n_epic>=100)
endpoints = endpoint_defs$endpoint

In [15]:
today = experiment

In [16]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major=element_line()))

In [17]:
endpoint_map = endpoint_defs$phecode_string
names(endpoint_map) =  endpoint_defs$endpoint
endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint

In [18]:
endpoint_selection = c(
    # generally very important
    "phecode_202", # Diabetes mellitus
    "phecode_401",	#Hypertension"  
    "phecode_404", # Ischemic heart disease   
    "phecode_404-1", # Myocardial infarction [Heart attack]
    "phecode_431-11", # Cerebral infarction [Ischemic stroke]
    "phecode_424", # Heart failure

    
    "phecode_059-1", # COVID 19
    "phecode_468", # Pneumonia
    "phecode_474", # Chronic obstructive pulmonary disease [COPD]
      
    "phecode_286-2", #	Major depressive disorder
    "phecode_324-11", #Parkinson's Disease
    "phecode_328", # Dementias and cerebral degeneration

    
    "phecode_164", # Anemia
    "phecode_726-1", # Osteoporosis
    "phecode_371", # Cataract
    "phecode_374-42", # Diabetic retinopathy
    "phecode_374-5", # Macular degeneration
    "phecode_375-1", # Glaucoma
    
    
    "phecode_103", # Malignant neoplasm of the skin
    "phecode_101", # Malignant neoplasm of the digestive organs
    "phecode_102", # LUNG CANCER
    
    "phecode_583", # Chronic kidney disease    
    "phecode_542", # Chronic liver disease and sequelae    
    "OMOP_4306655" # All-Cause Death
    
    # also generally important and relevant
    #"phecode_440-3", # Pulmonary embolism
    #"phecode_468-1",	#Viral pneumonia
#     "phecode_460-2",	#Acute lower respiratory infection
    #"phecode_388" # Blindness and low vision
      # generally important and fun to check
   # "phecode_374-3", # Retinal vascular changes and occlusions
    #"phecode_665", # Psoriasis
#     "phecode_121", # Leukemia
    # important for eye
#     "phecode_705-1", # Rheumatoid arthritis
)

endpoints_common = c(
   'phecode_164', #Anemia
 'phecode_705-1', #Rheumatoid arthritis
 'phecode_328', #Dementias and cerebral degeneration
 'phecode_328-1', #Alzheimer's disease
 'phecode_401', #Hypertension
 'phecode_202', #Diabetes mellitus
 'phecode_416-21', #Atrial fibrillation
 'phecode_404-1', #Myocardial infarction [Heart attack]
 'phecode_424', #Heart failure
 'phecode_468', #Pneumonia
 'phecode_474', #Chronic obstructive pulmonary disease [COPD]
 'phecode_583', #Chronic kidney disease
 'OMOP_4306655' #All-Cause Death
    )
    
endpoints_cardio = c(
    'phecode_438-11',   #  "Abdominal aortic aneurysm",
    'phecode_440-3',#  "Pulmonary embolism", # intervention
    'phecode_413-21',#  "Aortic stenosis", # intervention
    'phecode_400'#  "Rheumatic fever and chronic rheumatic heart diseases",	
)

endpoints_eye= c(
    'phecode_374-5', #Macular degeneration
 'phecode_374-51', #Age-related macular degeneration
 'phecode_374-42', #Diabetic retinopathy
 'phecode_371', #Cataract
 'phecode_388', #Blindness and low vision
 'phecode_367-5', #Uveitis
 'phecode_389-1' #Ocular pain
)

In [19]:
endpoint_defs = endpoint_defs %>% 
    mutate(name = phecode_string) %>%
    mutate(name = 
           case_when( 
               phecode_string == "Myocardial infarction [Heart attack]"~"Myocardial infarction",
               phecode_string == "Cerebral infarction [Ischemic stroke]"~"Ischemic stroke",
               phecode_string == "Chronic obstructive pulmonary disease [COPD]"~"Chronic obstructive pulmonary disease",
               phecode_string == "Mitral valve insufficiency"~"Mitral insufficiency",
               phecode_string == "Parkinson's disease (Primary)"~"Parkinson's disease",
               phecode_string == "Suicide ideation and attempt or self harm"~"Suicide attempt",
               phecode_string == "Ischemic heart disease"~"Coronary heart disease",
               phecode_string == "Chronic kidney disease"~"Chronic kidney disease",
               phecode_string == "Rheumatic fever and chronic rheumatic heart diseases"~"Rheumatic heart disease",
               phecode_string == "Abdominal aortic aneurysm"~"Abdominal aortic aneurysm",
                  TRUE ~ name)
           )
            
endpoint_map = endpoint_defs$name
names(endpoint_map) =  endpoint_defs$endpoint
#endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint
endpoint_order = endpoint_selection

In [20]:
endpoints_cardio = c(
    'phecode_431-11', #  "Cerebral infarction [Ischemic stroke]",
    'phecode_404', #  "Ischemic heart disease",
    'phecode_404-1', #  "Myocardial infarction [Heart attack]", # intervention
    'phecode_424', #  "Heart failure", # intervention
     'OMOP_4306655', #  "All-Cause Death", # intervention
    'phecode_420' #  "Cardiac arrest", # intervention
   
)

## Load data

# Load Benchmarks

In [22]:
name = "hrs_endpoints"
hrs_endpoints = arrow::read_feather(glue("{experiment_path}/{name}.feather"), 
                                    col_select=c("endpoint", "score", "partition", "hrs_ret", "hrs_ret_age", "hrs_ret_sex")) %>% 
    left_join(endpoint_defs) %>% filter(n_epic>=100)

Joining with `by = join_by(endpoint)`


In [23]:
hrs_endpoints

endpoint,score,partition,hrs_ret,hrs_ret_age,hrs_ret_sex,n_epic,eligable,n,freq,phecode,phecode_string,phecode_category,sex,name
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
OMOP_4306655,ASCVD+Retina,0,1.463347,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,1,1.437625,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,2,1.484841,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,3,1.464765,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,4,1.463176,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,5,1.397219,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,6,1.424078,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,7,1.491518,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,8,1.413272,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death


In [24]:
hrs = hrs_endpoints %>% 
    group_by(endpoint, score, phecode_string, phecode_category) %>% 
    #filter(score %in% c("Retina", "Age+Sex+Retina", "Age+Sex+Retina+I(Age*ret)")) %>%
    #mutate(endpoint = factor(endpoint, levels=endpoints_sorted)) %>%
    #mutate(phecode_category = factor(phecode_category, levels=pcats_sorted)) %>%
    select(endpoint, score, hrs_ret, hrs_ret_age, hrs_ret_sex, phecode_string, phecode_category) %>%
    ungroup()
hrs

endpoint,score,hrs_ret,hrs_ret_age,hrs_ret_sex,phecode_string,phecode_category
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
OMOP_4306655,ASCVD+Retina,1.463347,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.437625,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.484841,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.464765,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.463176,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.397219,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.424078,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.491518,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.413272,NA,NA,All-Cause Death,Death


In [25]:
unique(hrs$score)

[1] "ASCVD+Retina"   "Age+Sex+Retina" "Retina"         "SCORE2+Retina"

In [26]:
purrr::map_dfr(integer(100), ~ hrs %>% group_by(endpoint, score) %>% sample_frac(1, replace=TRUE), .id="i") -> temp_hrs

In [27]:
library(ggdist)

In [28]:
temp_hrs_ret = temp_hrs %>% group_by(endpoint, score, phecode_string, phecode_category) %>% median_qi(hrs_ret, hrs_ret_age, hrs_ret_sex, na.rm=TRUE)

In [34]:
temp_hrs_linear = temp_hrs %>% ungroup() %>%
    filter(score %in% c("Retina",
                        "Age+Sex+Retina"
#                        "SCORE2+Retina",
#                        "ASCVD+Retina",
#                        "QRISK3+Retina"
                       )) %>%
    group_by(endpoint, score, phecode_string, phecode_category) %>% 
    median_qi(`hrs_ret`, na.rm=TRUE) %>% rename(hr = `hrs_ret`) %>%
    mutate(hr = round(hr, 2), .lower=round(.lower, 2), .upper=round(.upper, 2)) %>%#c(hr, .lower, .upper), round, 1) %>%
    mutate(HR_Retina = as.character(glue("{hr} ({.lower}, {.upper})"))) %>% 
    select(endpoint, score, phecode_string, phecode_category, HR_Retina) %>%
    pivot_wider(names_from="score", values_from="HR_Retina") %>% 
    select(all_of(c("phecode_category", "endpoint", "phecode_string", "Retina", 'Age+Sex+Retina'
#                     "SCORE2+Retina", "ASCVD+Retina","QRISK3+Retina"
                   ))) %>% 
    #mutate(endpoint = factor(endpoint, levels = endpoint_order)) %>% 
    arrange(endpoint) %>%
    #mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>%
    ungroup()
    
temp_hrs_linear

phecode_category,endpoint,phecode_string,Retina,Age+Sex+Retina
<chr>,<chr>,<chr>,<chr>,<chr>
Death,OMOP_4306655,All-Cause Death,"3.26 (3.16, 3.6)","1.51 (1.37, 1.7)"
ID,phecode_089,Infections,"50.53 (43.82, 57.84)","5.33 (4.36, 6.49)"
ID,phecode_089-1,Bacterial infections,"4.28 (3.95, 4.71)","1.49 (1.3, 1.63)"
ID,phecode_089-2,Viral infections,"92.13 (47.77, 158.61)","20.64 (10.55, 41.57)"
ID,phecode_089-3,Fungal infections,"49.09 (22.95, 79.24)","16.29 (4.57, 26.49)"
ID,phecode_092,"Bacteremia, Sepsis, and SIRS","2.2 (2.06, 2.36)","1.25 (1.15, 1.37)"
ID,phecode_092-2,Sepsis,"2.24 (2.08, 2.39)","1.26 (1.15, 1.38)"
Neoplasms,phecode_101,Malignant neoplasm of the digestive organs,"2.06 (1.83, 2.28)","1.11 (0.95, 1.15)"
Neoplasms,phecode_101-4,Malignant neoplasm of the colon and rectum,"1.98 (1.81, 2.18)","1.07 (0.98, 1.12)"


In [39]:
ukb = fread("SupplTables - ST6_ HazardRatiosAll.csv")

In [40]:
ukb

endpoint,phecode_string,ukb_unadjusted,ukb_agesex
<chr>,<chr>,<chr>,<chr>
phecode_701-1,Osteomyelitis,"2.86 (0.01, 3.12)","0 (0, 3.03)"
phecode_701,"Osteomyelitis, periostitis, and other infections involving bone","2.92 (2.61, 3.15)","0.02 (0, 3.06)"
phecode_979,Transplated organ,"2.9 (0, 3.22)","0.07 (0, 3.66)"
phecode_142,Lump or mass in breast or nonspecific abnormal breast exam,"3.99 (3.55, 4.45)","1.12 (1.07, 1.17)"
phecode_520-11,Inguinal hernia,"4.06 (3.74, 4.88)","1.13 (1.09, 1.18)"
phecode_608,Other disorders of male genital organs,"1.21 (1.16, 1.29)","1.14 (1.11, 1.18)"
phecode_142-1,Lump or mass in breast,"3.53 (3.1, 3.8)","1.17 (1.11, 1.24)"
phecode_613-7,Other signs and symptoms in breast,"1.2 (1.06, 1.29)","1.18 (1.06, 1.29)"
phecode_613,Other nonmalignant breast conditions,"1.35 (1.3, 1.5)","1.18 (1.14, 1.28)"


In [44]:
epic = temp_hrs_linear %>% select(-phecode_category, -phecode_string)
colnames(epic) = c("endpoint", "epic_unadjusted", "epic_agesex")
epic

endpoint,epic_unadjusted,epic_agesex
<chr>,<chr>,<chr>
OMOP_4306655,"3.26 (3.16, 3.6)","1.51 (1.37, 1.7)"
phecode_089,"50.53 (43.82, 57.84)","5.33 (4.36, 6.49)"
phecode_089-1,"4.28 (3.95, 4.71)","1.49 (1.3, 1.63)"
phecode_089-2,"92.13 (47.77, 158.61)","20.64 (10.55, 41.57)"
phecode_089-3,"49.09 (22.95, 79.24)","16.29 (4.57, 26.49)"
phecode_092,"2.2 (2.06, 2.36)","1.25 (1.15, 1.37)"
phecode_092-2,"2.24 (2.08, 2.39)","1.26 (1.15, 1.38)"
phecode_101,"2.06 (1.83, 2.28)","1.11 (0.95, 1.15)"
phecode_101-4,"1.98 (1.81, 2.18)","1.07 (0.98, 1.12)"


In [48]:
st6 = ukb %>% left_join(epic, by="endpoint") %>% as_tibble() %>% arrange(endpoint)

In [49]:
st6_clean <- st6 %>%
  mutate(across(everything(), ~ replace_na(., ""), .names = "{col}"))

In [50]:
st6_clean %>% write_csv("outputs/SupplTable6_ukbepic_merged.csv")

In [32]:
# new, after bugfix
temp_hrs_linear %>% filter(endpoint %in% endpoint_selection) %>% select(-phecode_category)

endpoint,phecode_string,Retina,Age+Sex+Retina
<chr>,<chr>,<glue>,<glue>
OMOP_4306655,All-Cause Death,"3.26 (3.16, 3.6)","1.51 (1.37, 1.7)"
phecode_101,Malignant neoplasm of the digestive organs,"2.06 (1.83, 2.28)","1.11 (0.95, 1.15)"
phecode_103,Malignant neoplasm of the skin,"3.66 (3.3, 3.86)","1.19 (1.1, 1.39)"
phecode_164,Anemia,"2.39 (2.28, 2.5)","1.37 (1.27, 1.43)"
phecode_202,Diabetes mellitus,"3.38 (2.92, 3.55)","2.77 (2.33, 2.91)"
phecode_286-2,Major depressive disorder,"2.37 (1.99, 2.77)","1.65 (1.43, 1.97)"
phecode_328,Dementias and cerebral degeneration,"3.09 (3.02, 3.35)","1.07 (1.01, 1.17)"
phecode_371,Cataract,"3.74 (3.63, 3.8)","3.11 (3.03, 3.2)"
phecode_401,Hypertension,"3.68 (3.6, 3.85)","2.01 (1.95, 2.14)"


In [44]:
temp_hrs_linear %>% 
    filter(phecode_string %in% tmp_sign$phecode_string) %>%
    select(-phecode_category) %>%
    write_csv("outputs/SupplTable2_HRsLinearSIGNIFICANT_CropRatio-0.66.csv")